In [7]:
import cv2
import numpy as np
import pickle
import os
import sqlite3
import random
import pyttsx3
from threading import Thread
from sklearn.utils import shuffle
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.models import load_model

In [ ]:
#Defining the camera space 
def build_squares():
    x,y,w,h = 420,140,10,10 #Dimension of square
    d = 10 #
    imgCrop = None #Intialize the cropping of image
    for i in range(10): #
        for j in range(5): #
            if np.any(imgCrop == None): #if any imgcrop is empty do nothing continue
                imgCrop = img[y:y+h,x:x+w] #new image it capture in the range specified 
            else:
                imgCrop = np.vstack((imgCrop,img[y:y+h,x:x+w])) #Take a sequence of arrays and stack them vertically to make a single array
                #for more information https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.vstack.html
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
            #rectangle formed of the specified dimension and draw a green rectangle at the top-right corner of image and 2 is thickness.
            x += w+d 
        x = 420
        y += h+d
    return imgCrop

In [ ]:
def get_hand_hist():
    cam = cv2.VideoCapture(1) #Capture of image from index 1
    if cam.read()[0]==False:
        cam = cv2.VideoCapture(0) #capture of image from index 0
    x,y,w,h = 300,100,300,300 #Dimension of image specified
    flagPressedC,flagPressedS = False,False   #Intialize the keyboard S,C
    imgCrop = None #Intialize the Image Crop
    while True:
        img = cam.read()[1] #image read from index 1
        img = cv2.flip(img,1) #flip the image horizontal
        hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV) #convert the image from Blue,Green and Red to Hue,Saturation &Violet
        
        keypress = cv2.waitkey(1) #Each frame wait for 1 sec and then new frame starts
        if keypress == ord('c'):   #It converts the key pressed into integer
            hsvCrop = cv2.cvtColor(imgCrop,cv2.COLOR_BGR2HSV) #change the color into BGR2HSV
            flagPressedC = True #if the key pressed C
            hist = cv2.calcHist([hsvCrop],[0,1],None,[180,256],[0,180,0,256]) #calulate histogram 
            cv2.normalize(hist,hist,0,255,cv2.NORM_MINMAX) #normalize the images using min-max 
        elif keypress == ord('s'): #convert 'S' into numerical
            flagPressedS = True #if the button s is pressed
            break
        if flagPressedC: #if keyword 'c' is pressed
            dst = cv2.calcBackProject([hsv],[0,1],hist,[0,180,256],1) # It is used for image segmentation or finding objects of interest in an image.
            disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10))
            cv2.filter2D(dst,-1,disc,dst)
            blur = cv2.GaussianBlur(dst,(11,11),0)#Bluring the image using GaussianBlur 
            blur = cv2.medianBlur(blur,15) #Blurring the image using median Blur
            ret,thresh = cv2.thershold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU) #Set threshold to filter out specified image type
            thresh = cv2.merge(thresh,thresh,thresh) #merging the different images into one
            res = cv2.bitwise_and(img,thresh) #Calculates the per-element bit-wise conjunction of two arrays or an array and a scalar.
            cv2.imshow(thresh,thresh) #show the thershold image
        if not flagPressedS: #if the button 'S' is pressed
            img_crop = build_squares(img) #First run the function to capture image 
        cv2.imshow("Set hand histogram",img) #Show the image captured
    cam.release() #Release the capture 
    cv2.destroyAllWindows() #close the windows
    with open("hist","wb") as f:
        pickle.dumb(hist,f) #save the image dumb
        
get_hand_hist()

In [2]:
image_x,image_y = 50,50

In [3]:
def get_hand_hist():
    with open('hist','rb') as f:
        hist = pickle.load(f)
    return hist

In [4]:
def init_create_folder_database():
    if not os.path.exists("gestures"):
        os.mkdir("gestures")
    if not os.path.exists("guesture_db.db"):
        conn = sqlite3.connect("gesture_db.db")
        create_table_cmd = "CREATE TABLE gesture ( g_id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, g_name TEXT NOT NULL )"
        conn.execute(create_table_cmd)
        conn.commit()

In [5]:
def create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)

In [6]:
def create_empty_images():
    create_folder("guestures/"+ folder_name)
    black = np.zeros(shape = (image_x,image_y,1),dtype=np.uint8)
    for i in ranges(n_images):
        cv2.imwrite('guestures/'+folder_name+'/'+str(i+1)+'.jpg',black) 
#save images in black and white in the folder and with name start 1 to till last and save in .jpg format

In [13]:
def store_in_db(g_id,g_name):
    conn = sqlite3.connect("gesture_db.db") #connect with sqlite database we have created above bloack
    cmd = "INSERT INTO gesture (g_id, g_name) VALUES (%s, \'%s\')" % (g_id, g_name) #insert the g_id and g_name into sqlite database 
    try:
        conn.execute(cmd) #trying the above to connect using the try function
    except sqlite3.IntrgrityError: #except the following error
            choice = input("g_id already exists. Want to change the record? (y/n): ")
            if choice.lower == 'y':
                cmd = "UPDATE gesture SET g_name = \'%s\' WHERE g_id = %s" % (g_name, g_id)
                conn.execute(cmd)
            else:
                print('Doing nothing.....')
                return
#store image in sqlite database by coonecting the sqlite database where it store image id and image name using try and
#exception function if image exists in database then it will ask to upddate or not

In [ ]:
def store_images(g_id):
    total_pics = 1200 #total number of pictures going to save
    if g_id == str(0): #if iamge_id is zero
        create_empty_images("0",total_pics) #then create number of empty images from zero to 1200
    hist = get_hand_hist() #**get history of hand I guess it's MNIST data avaliable in kaggle
    #For more information https://www.kaggle.com/c/digit-recognizer/data
    cam = cv2.VideoCapture(1) #Used opencv library to capture image
    #To capture a video, you need to create a VideoCapture object. Its argument can be either the device index or
    #the name of a video file. Device index is just the number to specify which camera. Normally one camera
    #will be connected (as in my case). So I simply pass 0 (or -1). You can select the second camera by passing
    #1 and so on
    if cam.read()[0] == False: #if camera doesnot read from the previous camera index then capture camera index 0
        cam = cv2.VideoCapture(0)
    x,y,w,h = 300,100,300,300  # it's length&width of image x,y,w,h 
    create_folder("gestures"+str(g_id)) # create a folder gestures and save all the files 
    pic_no = 0 #Initialise the pic_no variable
    flag_start_capturing = False #Initialise frame caputing image
    frames = 0 #initalise frame
    while True: #when it read the image
        img = cam.read()[1] #image capture by videocapture read and store in Img
        img = cv2.flip(img,1) #flip every image in horizontal direction '1' nd '0' in vertical images
        imgHSV = cv2.ctvColor(img,cv2.COLOR_BGR2HSV) #Convert (BGR) Blue,Green and Red into (HSV) Hue,Saturation and Chroma
        #For more information https://stackoverflow.com/questions/17239253/opencv-bgr2hsv-creates-lots-of-artifacts
        dst = cv2.calcBackProject([imgHSV],[0,1],hist,[0,180,0,256],1) #Calculates the back projection of a histogram.
        #For more information https://docs.opencv.org/2.4/modules/imgproc/doc/histograms.html?highlight=calcbackproject
        disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10)) #For elliptical/circular shaped kernels shape.
        #For more information https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html
        cv2.filter2D(dst,-1,disc,dst) # we are try an averaging filter on an image
        #For more information https://docs.opencv.org/3.1.0/d4/d13/tutorial_py_filtering.html
        blur = cv2.GaussianBlur(dst,(11,11),0) #to blurr image
        #For more information https://docs.opencv.org/3.1.0/d4/d13/tutorial_py_filtering.html
        #To make the image noise free 
        blur = cv2.medianBlur(blur,15)# to blurr image by another method
        #takes median of all the pixels under kernel area and central element is replaced with this median value
        #Another method to make the image noise free
        #For more information https://docs.opencv.org/3.1.0/d4/d13/tutorial_py_filtering.html
        thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1] #thershold to define filter out the images of grey scale image
        #For more information https://docs.opencv.org/3.4.0/d7/d4d/tutorial_py_thresholding.html
        thresh = cv2.merge(thresh,thresh,thresh) # Merging of images 
        #For more information https://docs.opencv.org/3.1.0/d3/df2/tutorial_py_basic_ops.html
        thresh = cv2.cvtColor(thresh,cv2.COLOR_BGR2GRAY) #converting image into GRAY
        contours = cv2.findContours(trhresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)[1] #a curve joining all the continuous points (along the boundary), having same color or intensity.
        #For more information https://docs.opencv.org/3.1.0/d4/d73/tutorial_py_contours_begin.html
        if len(contours)>0: #if length(contours greater than zero)
            contour = max(contours,key = cv2.contourArea)# maximum between contours and contourArea
            if cv2.contourArea(contour)>10000 and  frames>50: #number of frames greater than 50 and ContourArea greater than 1000
                x1,y1,w1,h1 = cv2.boundingRect(contour) #convert the image into rectange 
                pic_no += 1 #calculate number of image
                save_img = thresh[y1:y1+h1,x1:x1+w1] #save image by filtering threshold and define the width and length
                if w1 > h1: #if width is greater than height then copy border below formula
                    save_img = cv2.copyMakeBorder(save_img,int((w1-h1)/2),int((w1-h1)/2),0,0,cv2.BORDER_CONSTANT,(0,0,0))
                elif h1 > w1: #if height is greater than width then copyborder below formula
                    save_img = cv2.copyMakeBorder(save_img,int((h1-w1)/2),int((h1-w1)/2),0,0,cv2.BORDER_CONSTANT,(0,0,0))
                save_img = cv2.resize(save_img,(image_x,image_y)) #save entire  image above specified dimension 
                cv2.putText(img,'Capturing....',(30,60),cv2.FONT_HERSHEY_TRIPLEX,2,(127,255,255))#putText renders the specified text string in the image.
                #For more information https://docs.opencv.org/3.1.0/d6/d6e/group__imgproc__draw.html#ga5126f47f883d730f633d74f07456c576
                cv2.imwrite("guesture/"+str(g_id)+"/"+str(pic_no)+".jpg",save_img) #saves the image to the specified file. 
                #For more information https://docs.opencv.org/3.0-beta/modules/imgcodecs/doc/reading_and_writing_images.html#bool imwrite(const String& filename, InputArray img, const vector<int>& params)                
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2) #Save images into rectangle specified heigh and width and specified image pixek 
        cv2.putText(img,str(pic_no),(30,400),cv2.FONT_HERSHEY_TRIPLEX,1.5,(127,127,255))
        cv2.imshow("Capturing guesture",img) #show the images captured
        cv2.imshow("thresh",thresh) #show the images after filtering out from thershold 
        keypress = cv2.waitkey(1) #displays the image for specified milliseconds
        #For example, waitKey(0) will display the window infinitely until any keypress (it is suitable for
        #image display). waitKey(25) will display a frame for 25 ms, after which display will be automatically
        #closed. (If you put it in a loop to read videos, it will display the video frame-by-frame)
        if keypress == ord('c'): #Given a string of length one, return an integer representing the Unicode code point
            #of the character when the argument is a unicode object, or the value of the byte when the argument is an 
            #8-bit string
            if flag_start_capturing == False:
                #if camera is not capturing image then start capture image
                flag_start_capturing = True
            else:
                flag_start_capturing = False 
                #if its's not capturing the image
                frames = 0  #then number of frames is zero
        if flag_start_captring == True: #if flag start capturing then calculate number of frames and count number of total pictures 
            frame += 1
        if pic_no == total_pics: #if total picture equivalent to define total pictures then break coe out of loop
            break
init_create_folder_database() # intialise the function
g_id = input("Enter gesture no.:") # input gesture no by showing hand 
g_name  = input("Enter gesture name/text:") #Input gesture name  
store_in_db(g_id,g_name) #store image into sqlite database
store_images(g_id) #store image in the folder satisfying the specified condition in the function

In [ ]:
#Labelling the image 
def pickle_images_labels():
    gest_folder = "gestures" #folder where image are saved
    images_labels = [] #Intialize the array Labeled image 
    images = [] #intialize images array
    labels = [] #intialize labels array
    for g_id in os.listdir(gest_folder):   #from the gest_folder
        for i in range(1200):   #from the 1200 intial images 
            img = cv2.imread(gest_folder+"/"+g_id+"/"+str(i+1)".jpg",0) #Reading the images by opencv
            if np.any(img == None):    #if any image comes to none due to threshold
                continue
            images_labels.append((np.array(img,dtype=np.float32),int(g_id))) #otherwise append those image in image_labels as numpy array
    return images_lables            

In [ ]:
def split_images_labels(images_labels):
    images = []
    labels = []
    for(image,label) in image_labels:
        images.append(image) #storing all the image in image array
        labels.append(label) #storing all the lables in labels array
    return images,labels

In [ ]:
images_labels = pickle_images_labels() #save the results of function into images_labels
images_labels = shuffle(shuffle(image_labels)) #Shuffle the results of lables
images,labels = split_images_labels(images_labels) #sliting images and lables
print("Length of images labels",len(images_labels)) #Length of image_labels

In [ ]:
train_images = images[:int(5/6*len(images))] #Spliting the images into train_image from image file
print("Length of train images",len(train_images)) #Finding the lenth of train_images
#The data format used by pickle is Python-specific. This has the advantage that there are no restrictions imposed
#by external standards such as JSON or XDR
#For more imformation https://docs.python.org/3/library/pickle.html#data-stream-format
with open("train_images","wb") as f:  #open the train images 
    pickle.dumb(train_images,f)  #dump the images in pickle format(i.e. pythonic format)
del train_images #delete train_image as it is of no use because we are going to use pickle one for future

In [ ]:
train_labels = labels[:int(5/6*len(labels))] #Spliting of train_labels from the labels
print("Length of train labels",len(train_labels)) #print the length of train_labels
with open("train_labels","wb") as f: #opening of file train_labels
    pickle.dumb(train_labels,f) #dumb file of train_labels
del train_labels #delete of train_labels

In [ ]:
test_images = images[int(5/6*len(images)):] #splittig of test_images from images
print("Length of test images",len(test_images)) #length of test images
with open("test_images","wb") as f: #open the test images 
    pickle.dumb(test_images,f) #dumb the test file into pickle format
del test_images #delete of test_images

In [ ]:
test_labels = labels[int(5/6*len(labels)):] #splitting of test_labels from labels
print("Length of test Labels",len(test_labels)) #length of test_labels
with open("test_labels","wb") as f: #open the test_labels file
    pickle.dumb(test_labels,f) #dumb  of the test_labels into pickle format
del test_labels #delete of test_labels

In [ ]:
def get_image_size():
    img = cv2.imread('gesture/0/100.jpg',0) #Read the images from the gven floder
    return img.shape #give the value of total dimension i.e. nrow * ncol

In [ ]:
gestures = os.listdir('gestures/') #list all the files in the given folder
gestures.sort(key = int) #sort the files in ascending order
begin_index = 0 #initialize the begin index 
end_index = 5 #initialize the end index with 5 minimum
image_x, image_y = get_image_size() #store image into image_x & image_y

In [ ]:
if len(gestures)%5 !=0:           #splitting rows and the condition if number of rows not multple of 5 then calculate rows= (rows/5+1) 
    rows = int(len(gestures)/5)+1
else:
    rows = int(len(gestures)/5) #otherwise spliting of rows into row/5

In [ ]:
full_img = None #Initialise the rows
for i in range(rows):
    col_img = None #Intialise the image in the given rows
    for j in range(begin_index,end_index): #Indexing the image 
        img_path = "gestures%s/%d.jpg" % (j, random.randint(1,1200)) #select the image randomly from 1 to 1200
        img = cv2.imread(img,0) #Read the images from index 0
        if np.any(img == None): #if any image is not their
            img = np.zeros((image_y,image_x),dtype=np.unit8) #Create empty array with imag dimension np.unit8
        if np.any(col_img == None): #if any col_img is not their then col_img is equal to img
            col_img = img
        else:
            col_img = np.hstack((col_img,img)) #Otherwise stack the image horizontally
    begin_index += 5 #Next loop start cnsidering next 5 images
    end_index += 5 #similarly the end index increment value 05
    if np.any(full_img == None): #if any full_img is not there then full_img = col_img
        full_img = col_img
    else:
        full_img = np.vstack((full_img,col_img)) #stacking of full_img vrtically by index of col_img

In [ ]:
cv2.imshow('gestures',full_img) #show all the images it read
cv2.waitkey(1) #Wait for 1 sec to next image to show 

In [ ]:
#"th" format means that the convolutional kernels will have the shape (depth, input_depth, rows, cols)
#"tf" format means that the convolutional kernels will have the shape (rows, cols, input_depth, depth)
#Therefore you can convert from the former to the later via np.transpose(x, (2, 3, 1, 0)) where x is the value of 
#the convolution kernel.
#For more information https://stackoverflow.com/questions/39547279/loading-weights-in-th-format-when-keras-is-set-to-tf-format
K.set_image_dim_ordering('tf') 

In [ ]:
#In TensorFlow 0.12+, per this issue, you can now control logging via the environmental variable called 
#TF_CPP_MIN_LOG_LEVEL; it defaults to 0 (all logs shown), but can be set to 1 to filter out INFO logs, 2 to 
#additionally filter out WARNING logs, and 3 to additionally filter out ERROR logs.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [ ]:
def get_image_size():
    img = cv2.imread('gestures/0/100.jpg',0) #Reading of image and convert into array of zero and shape of (50,50)
    return img.shape

In [ ]:
def get_num_of_classes():
    return len(os.listdir('gestures/')) #number of folders or files in the given directory

In [ ]:
image_x,image_y = get_image_size() #seperating image_x and image_y from the images read

In [ ]:
def cnn_model():
    #For more information regarding model https://keras.io/layers/core/
    num_of_classes = get_num_of_classes() # number of classes is equal to number of files in the folder
    model = Sequential() #Calling of sequential model in keras
    model.add(conv2D(32,(5,5),input_shape = (image_x,image_y,1),activation = 'sigmoid')) #convoluional model applied input dimension and acivation specified
    model.add(MaxPooling2D(pool_size = (2,2),strides = (2,2),padding = 'same')) #no. of strides to be taken and paddig for corner pixels
    #For more information https://keras.io/layers/core/#dense
    model.add(Flatten()) #https://stackoverflow.com/questions/43237124/role-of-flatten-in-keras
    model.add(Dense(1024,activation = 'relu')) #relu activation 
    model.add(Dropout(0.6)) #drop out all those hidden layer whic has value less than 0.6
    model.add(Dense(num_of_classes , activation = 'softmax')) #atlast non-linear using activaton softmax
    sgd = optimizers.SGD(lr = 1e-2) #optimized the function using Stochastic Gradient Descent and learning rate 0.01
    model.compile(loss='categorical_crossentropy',optimizers=sgd,metrics=['accuracy'])
    filepath = 'cnn_model_keras2.h5' #saving the model 
    checkpoint1 = ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=True,mode='max') #save the best results
    checkpoint2 = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min') #another level of checkpoint of saving the best result
    callbacks_list = [checkpoint1] 
    return model,callback_list #Retrun the result of model and callbacks_list is the best variable we have selected

In [ ]:
def train():
    with open("train_images","rb") as f: #read the train_images
        train_images = np.array(pickle.dumb(f)) #dump the images as array in pickle format
    with open("train_labels","rb") as f: #read the train_labels
        train_labels = np.array(pickle.dumb(f),dtype = np.int32) #dump the labels as array of int 32 in pickle format
    with open("test_images","rb") as f: #read the test_images
        test_images = np.array(pickle.dumb(f)) #dump the test images in the picke format as an array 
    with open("test_labels","rb") as f: #read the test labels
        test_labels = np.array(pickle.dumb(f),dtype = np.int32) #dump the test_labels in array of int32 in pickle format
    
    train_images = np.reshape(train_images, (train.shape[0],image_x,image_y,1)) #reshaping the train_images,so that all the images into same shape which can be ustilised in model 
    test_images  = np.reshape(test_images,(test.shape[0],image_x,image_y,1)) #reshaping the test_images
    train_labels = np_utils.to_categorical(train_labels)  #converting tain_labels into categorical
    test_labels = np_utils.to_categorical(test_labels) #converting test_labels into categorical
    
    model,callback_lists = cnn_model() #Result of above architacture uses in cnn_model() function
     model.fit(train_images,train_labels, validation_data = (test_images,test_labels),epoch=50,batch_size=100,callbacks=callbacks_list) #training the above model architecture in train_images,train_labels 
    scores = model.evaluate(test_images,test_labels,verbose=0) #calculating the score of model 
    print("CNN Error: %.2f%%" % (100- scores[1]*100)) #showing the error rate i.e. percentage of variatin not explained by model
    model.save('cnn_model_keras.h5') #save the model results 

train() #start the train function by calling it seperately
K.clear_session(); #clear the session

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Above explained
tf.logging.set_verbosity(tf.logging.ERROR) #Above explained
classifier = tf.estimator.Estimator(model_dir="tmp/cnn_model2", model_fn=cnn_model_fn) #loaidng of model by tensorflow
prediction = None #Initialize the prediction column
model = load_model('cnn_model_keras2.h5') #loding of model

In [ ]:
def get_image_size():
    img = cv2.imread('gestures/0/100.jpg',0) #reading of image
    return img.shape #return the image shape

In [ ]:
image_x,image_y = get_image_size() #Seperating the dimension into x and y

In [ ]:
def tf_image_process(img):
    img = cv2.resize(img,(image_x,image_y)) #resizing image with specified dimension
    img = np.array(img,dtype = n.float32) #converting the image into array with uint32 
    np_array = np.array(img)
    return np_array

In [ ]:
def tf_image(classifier,image):
    global prediction #globaly used prediction for one purpose
    processed_array = tf_process_image(image) #processing of image using above function 
    process_input_fn = tf.estimators.numpy_input_fn(x={"x":processed_array},shuffle = False) #seperating the variables x
    pred = classifier.predict(input_fn = pred_input_fn) #predicting y variable from the model
    prediction = next(pred) #The next() returns the next item from the iterator.
    print(prediction)

In [ ]:
def keras_process_image(img):
    img = cv2.resize(img,(image_x,image_y)) #resizing the image pass through keras model 
    img = np.array(img,dtype = np.float32) #converting image into arrays
    img = np.reshape(img,(1,image_x,image_y,1)) #rehshape image into specified dimension
    return img #return the img with new format, this is to standardise the images

In [ ]:
def keras_predict(model,image):
    processed = keras_process_image(image) #processed the image using above images
    pred_probab = model.predict(processed)[0] # predict the class_labels using the keras model
    pred_class = list(pred_probab).index(max(pred_probab)) #list the result and sort the result which has maximum probability in top
    return max(pred_probab),pred_class # return result of max of pred_probability and the pred_class

In [ ]:
def get_pred_text_from_db(pred_class):
    conn = sqlite3.connect('gesture_db.db') #connect with sqllite database where we stored images 
    cmd = "SELECT g_name FROM gesture WHERE g_id="+str(pred_class) #connect the pred_class with gestures_id and name 
    cursor = conn.execute(cmd) #execute the abve results
    for row in cursor: #consider the zeroth row every element
        return row[0]

In [ ]:
def split_sentence(text,num_of_words):
    list_words = text.split(" ") #spliting of each word with ('') used for tokenization
    length = len(list_words) #total number of words
    splitted_sentence = [] #Defined the empty matrice to store the splitted words of sentence
    b_index = 0 #Initialize the index of begining
    e_index = num_of_words #End of the index total number of words
    while length >0: #if total number of words greater  than zero
        part = "" #intialize the seperation using the part
        for word in list_words[b_index:e_index]: #considering all the words starting from b_index to e_index
            part = part + " "+ word #seperating each word(",")
            splitted_sentence.append(part) #adding to sentence matrix
            b_index += num_of_words #with each iteration b_index increases with number of words processed
            e_index += num_of_words #with each iteration e_index increases with number of words processed
            length -=  num_of_words #with each iteration length decreases with number of words processed
    return splitted_sentence #return the result of all the seperated sentence into one matrix

In [ ]:
def put_splitted_text_in_blackboard(blackboard,splitted_text):
    y = 200 #intialise the value of y with 200
    for text in splitted_text:
        cv2.putText(blackboard,text,(4,y),cv2.FONT_HERSHEY_TRIPLEX,2,(255,255,255)) #put the text into blackboard the font is defined 
        y += 50 #It is iteration of 50

In [ ]:
def get_hand_hist():
    with open("hist","rb") as f: #Read the hist file we have downloaded from kaggle famous MNIST data
        hist = pickle.load(f) #load into the system in pickle format
    return hist #Return the result in the above mentioned format

In [ ]:
def recognize():
    global prediction  #prediction has same purpose and meaning in the entire function
    cam = cv2.VideoCapture(1) #capture image into index 1
    if cam.read()[0] == False: #if the image is not Caputred at index 1(it means camera at index 1)
        cam = cv2.VideoCapture(0)#then capture the image at index 0
    hist = get_hand_hist() #call the results from above function
    x,y,w,h = 300,100,300,300 #defining the dimension of image
    while True:
        text = "" #intialize  the text
        img = cam.read()[1] #Readinng of image captured
        img = cv2.flip(img,1) #flip the image into horizontal
        imgCrop = img[y:y+h,x:x+w] #crop the image into new dimension
        imgHSV = cv2.cvtColor(img,cv2.COLOR_BGR2HSV) #changing the color BGR into HSV
        dst = cv2.calcBackProject([imgHSV],[0,1],hist,[0,180,0,256],1) #Removing the noise using BackProject
        disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10)) #Removing the nosie 
        cv2.filter2D(dst,-1,disc,dst) #Filtering the image and converting into 2D
        blur = cv2.GaussianBlur(dst,(11,11),0) #adding Gaussian Blur
        blur = cv2.medianBlur(blur,15) #Adding the median Blur
        thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1] #setting the thrshold to further process of image
        thresh = cv2.merge(thresh,thresh,thresh) #merge all the images after applying threshold 
        thresh = cv2.cvtcolor(thresh,cv2.COLOR_BGR2GRAY) #converting the new image into GRAY
        thresh = thresh[y:y+h,x:x+w] #setting the threshold in the dimension to standerdize
        contours = cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)[1] #finding the contours so that the all the similar points can be combined 
         if len(contours)>0: #if length of contour is greater than zero
            contour = max(contours,key = cv2.contourArea) #take those areas which has maximum assimilation of points
            if cv2.contourArea(contour) > 10000: #if contour Area greater than 10000
                x1,y1,w1,h1 = cv2.boundingRect(contour) #Bound it using the defined dimension
                save_img = thresh[y1:y1+h1,x1:x1+w1] #save image above specified dimension
                if w1 > h1: #if width greater than height,then save image inn the below configruation
                    save_img = cv2.copyMakeBorder(save_img,int((w1-h1)/2),int((w1-h1)/2),0,0,cv2.BORDER_CONSTANT,(0,0,0))
                elif h1 > w1: #if the height is greater than width,then save image into beow configruation
                    save_img = cv2.copyMakeBorder(save_img,int((h1-w1)/2),int((h1-w1)/2),0,0,cv2.BORDER_CONSTANT,(0,0,0))

                    pred_probab,pred_class = keras_predict(model,save_img) #predict the image labels
                    print(pred_class,pred_probab) #show the results
                    
                    if pred_probab * 100 > 80: # if pred_probability is greater than 80
                        text = get_pred_text_from_db(pred_class) #Then connect with sqlite database label it associate to give gesture_id
                        print(text) #print text
        blackboard = np.zeros((480,640,3),dtype = np.uint8) #defining the numpy arrays with zero and given configruation
        splitted_text = split_sentence(text,2) #split the text using split function and process it
        put_splitted_text_in_blackboard(blackboard,splitted_text) #then display the text into blackboard using the function
        cv2.putText(blackboard,text,(30,200),cv2.FONT_HERSHEY_TRIPLEX,1.3,(255,255,255)) #the text is processed in above function and put into specified format
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2) #display of image into rectange with given dimension
        res = np.hstack((img,blackboard)) #stack the images horizontally
        cv2.imshow("Recognise gesture",res)#show the images
        cv2.imshow("thresh",thresh) #show the images after process of threshold
        if cv2.waitkey(1) == ord('q'): #wait for 1 sec before displaying the next image
            break
keras_predict(model,np.zeros(50,50),dtype = np.uint8) #predict the class labels using the keras after processing of image
recognize() # after process of image and prediction of keras display the result by calling the given function

In [ ]:
#For more information pyttsx3 http://pyttsx3.readthedocs.io/en/latest/engine.html
engine = pyttsx3.init() #to activate speech driver
engine.setProperty('rate', 150)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #to avoid any type of warning or log error 
model = load_model('cnn_model_keras2.h5') #upload the keras model

In [ ]:
def get_hand_hist():
    with open("hist","rb") as f: #open the hist file which we have downloaded from the kaggle
        hist = pickle.load(f) #dump the hist file into pickle format
    return hist #return hist in the pickle format

In [ ]:
def get_image_size():
    img = cv2.imread('gestures/0/100.jpg',0) #read the image and covert into the array of zero
    return img.shape #return the shape of image

In [ ]:
image_x,image_y = get_image_size() #seperate x and y dimension of zero array image

In [ ]:
def keras_process_image(img):
    img = cv2.resize(img,(image_x,image_y)) #resize the image 
    img = np.array(img,dtype = uint8) #convert into array
    img = np.reshape(img,(1,image_x,image_y,1)) #reshape the image 
    return img #return the img after the above changes

In [ ]:
def keras_predict(model,image):
    processed = keras_process_image(image) #processed image accoring to above condition
    pred_probab = model.predict(processed)[0] #save the probability of image into different category
    pred_class = list(pred_probab).index(max(pred_probab)) #list the probability accoridng to value from max to min
    return max(pred_probab),pred_class #return max probability and pred_class

In [ ]:
def get_pred_text_from_db(pred_class):
    conn = sqlite3.connect('gesture_db.db') #connect to sqlite 
    cmd = "SELECT g_name FROM gesture WHERE g_id="+str(pred_class) #connect the pred_clas with image g_id
    cursor = conn.execute(cmd) #execute the command
    for row in cursor: #consider the first row f each image
        return row[0]

In [ ]:
def get_pred_from_contour(contour,thresh):
    x1,y1,w1,h1 = cv2.boundingRect(contour) #set boundary once the contour has finalized
    save_img = thresh[y1:y1+h1,x1:x1+w1] #save image acccording to new specifid dimension
    text = "" #intalize the text
    if w1 > h1 : #if width is greater than height then save the image in the below mentined specification
        save_img = cv2.copyMakeBorder(save_img,int((w1-h1)/2),int((w1-h1)/2),0,0,cv2.BORDER_CONSTANT,(0,0,0))
    elif h1 > w1: #if height is greater than width then save the image in the below mentioned specification
        save_img = cv2.copyMakeBorder(save_img,int((h1-w1)/2),int((h1-w1)/2),0,0,cv2.BORDER_CONSTANT,(0,0,0))
        pred_probab, pred_class = keras_predict(model,save_img) #new pred_probab and pred_class once image processed through keras and predict for future
        if pred_probab * 100 > 70: #if pred_prbobaility is greater than 70 to consider
            text = get_pred_text_from_db(pred_class) #then the text attached to pred_class processed throught given function and display result
        return text #return the text

In [ ]:
def get_operator(pred_text):
    try:
        pred_text = int(pred_text) #converting pred_text into integer
    except:
        return ""
    operator = ""
    if pred_text == 1: #if integer comes to 1 then the operator refer to '+' similarly for others
        operator = "+"
    if pred_text == 2:
        operator = "-"
    if pred_text == 3:
        operator = "*"
    if pred_text == 4:
        operator = "/"
    if pred_text == 5:
        operator = "%"
    if pred_text == 6:
        operator = "**"
    if pred_text == 7:
        operator = ">>"
    if pred_text == 8:
        operator = "<<"
    if pred_text == 9:
        operator = "&"
    if pred_text == 0:
        operator ="|"
    return operator #return the operator 

In [ ]:
hist = get_hand_hist() #read the image of hist
x,y,w,h = 300,100,300,300 #setting the dimension
is_voice_on = True #default voice is on

In [ ]:
def get_img_contour_thresh(img):
    img = cv2.flip(img,1) #flip the image horizontally
    imgHSV = cv2.cvtColor(img,cv2.COLOR_BGR2HSV) #convert image into bgr2hsv
    dst = cv2.calcBackProject([imgHSV],[0,1],hist,[0,180,0,256],1) #removing the noise
    disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10))#converting to elipse to remove sharpness
    cv2.filter2D(dst,-1,disc,dst) #converting into 2D
    blur = cv2.GaussianBlur(dst,(11,11),0) #adding Gaussian Blur
    blur = cv2.medainBlur(blur,15) #Adding median Blur
    thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1] #setting up thershold
    thresh = cv2.merge(thresh,thresh,thresh) #merging the images after setting up threshold
    thresh = cv2.cvtColor(thresh,cv2.COLOR_BGR2GRAY) #converting the threshold image into grey
    thresh = thresh[y:y+h,x:x+w] #setting up its dimenion
    contours = cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)[1] #using contour assimilate same points
    return img,contours,thresh   #return the results image ,contours,threshold image

In [ ]:
def say_text(text):
    if not is_voice_on: #if voice is not on then simply do nothing return
        return
    while engine._inloop:
        pass
    engine.say(text) # if it's on then speak the given text 
    engine.runAndWait()  #run the text in the above command

In [ ]:
def calculator_mode(cam):
    global is_voice_on   #is_voice_on has same meaning in the entire below function
    first,operator,second = "","","" #Intialise 
    pred_text = "" #prediction text initialize
    calc_text = "" #the text taken into consideration has initialised
    info = "Enter first number" 
    #for more information https://en.wikibooks.org/wiki/Python_Programming/Threading
    #https://www.geeksforgeeks.org/multithreading-python-set-1/
    Thread(target = say_text,args= (info,)).start() #whatever one input it simultaeneously activate say_text function to say the text    count_clear_frames = 0
    while True: #while the given function execute
        img = cam.read()[1] #read the image
        img,contours,thresh = get_img_cotour_thresh(img) #setting up the thrshold and contour
        old_pred_text = pred_text #old_text replace and new text with change pf frame
        if len(contours)>0: #if length of contour greater than zero
            contour = max(contours,key = cv2.contourArea) #take those area which has maximum contour Area
            if cv2.contourArea > 10000: #if contourArea greater than 10000
                pred_text = get_pred_from_contour(contour,thresh) #predict the text according to the given function conditon
                if old_pred_text == pred_text : #if previous text equal to next one then the count_same_frames increases
                    count_same_frames += 1
                else:
                    count_same_frames = 0 #otherwise the frame changes
                if pred_text == 'C': # if prediction text is 'C'
                    if count_same_frames > 5: #if the count of same frame greater than 5
                        count_same_frames = 0 #Initialise cont_same_frame to zero
                        first,second,operator,pred_text,calc_text = "","","","","" #initialize other variables
                        flag['first'],flag['operator'],flag['second'],flag['clear'] = False,False,False,False #take these variables false as default
                        info ='Enter first number'
                        Thread(target = say_text,args = (info,)).start() #then the read the new text once the new text enter
                elif pred_text = "Best of Luck" and count_same_frames > 15: #if pred text is in the wrtten and number of same frame exceeds 15
                    count_same_frame = 0 #initialize the count_same_frame to zero
                    if flag['clear']: #if the given box is clear
                        first,second,operator,pred_text,calc_text = "","","","","" #Initialize all the variables
                        flag['first'],flag['second'],flag['operator'],flag['clear'] = False,False,False,False #taking all the variables as false
                        info = 'Enter first number'
                        Thread(target=say_text,args=(info,)).start() #then enter the word and simultaeneously say text
                    elif second = "": #if its second
                        flag['second'] = True 
                        info = 'clear screen' #clear the screen
                        Thread(target = say_text,args = (info,)).start() #say the text which is going to input or predict
                        second = ""
                        flag['clear'] = True 
                        calc_text += "= " +str(eval(calc_text)) #evealuate the calculated text
                        if is_voice_on: #replace the word according to processed text
                            speech = calc_text
                            speech = speech.replace('-','minus ')
                            speech = speech.replace('/','divided by ')
                            speech = speech.replace('**','raised to the power')
                            speech = speech.replace('*','multiply')
                            speech = speech.replace('%','mod')
                            speech = speech.replace('>>','bitwise right shift')
                            speech = speech.replace('<<','bitwise left shift')
                            speech = speech.replace('&','bittwise and')
                            speech = speech.replace('|','bitwise or')
                        else first != '': #if first is not empty then input operator and say the text
                            flag['first'] = True
                            info = 'Enter operator'
                            Thread(target = say_text,args=(info,)).start()
                            first = ''
                elif pred_text != "Best of Luck " and pred_text.isnumeric(): #if pred text is not the mentioned one and its not numeric
                    if flag['first'] == False: #then first is false
                        if count_same_frames > 15: #if number same frame greater than 15
                            count_same_frame = 0 #Initialize the same frame
                            Thread(target = say_text, args = (pred_text,)).start() #say the predicted text
                            first += pred_text #then replace the word with pred text
                            calc_text += pred_text #Add the pred text into caluclation text
                    elif flag['operator'] == False: #if there is no operator
                        operator = get_operator(pred_text) #then convert into operator
                        if count_same_frames > 15: #if number of same frame greater than 15
                            count_same_frames = 0 #Initialize with 0
                            flag['operator'] = True #if there is operator
                            calc_text += operator #add the operator into process text
                            info = 'Enter second number' #enter the second number
                            Thread(target = say_text,args=(info,)).start() #say the text i.e. second number
                    elif flag['second'] == False: #if there is no second number
                        if count_same_frame >  15: #if number of same frame greater than 15
                            Thread(target=say_text,args=(pred_text,)).start() #say the text 
                            second += pred_text #put the predictive into second
                            calc_text += pred_text #Add the value into processed text
                            count_same_frame = 0 #number of same frame again intiialize to zero
    if count_clear_frames == 30: #if number of same frame greater than 30
            first,second,operator,pred_text,calc_text = "","","","","" #Intialize the variables
            flag['first'],flag['second'],flag['operator'],flag['clear']=False,False,False,False #set the variables False
            info = 'Enter first number' #enter the first number
            Thread(target = say_text,args=(info,)).start() #say the text whichever is entered
            count_clear_frames = 0 #Initialize the number of frames
        
        blackboard = np.zeros((480,640,3),dtype = uint8) #initialize the array
        cv2.putText(blackboard,"calculator mode",(100,50),cv2.FONT_HERSHEY_TRIPLEX,1.5,(255,0,0)) #processed  text format
        cv2.putText(blackboard,"predicted text-"+pred_text,(30,100),cv2.FONT_HERSHEY_TRIPLEX,1,(255,255,0)) #predicted text fprmat
        cv2.putText(blackboard,"operator"+operator,(30,240),cv2.FONT_HERSHEY_TRIPLEX,1,(255,255,127)) #operator format
        cv2.putText(blackboard,calc_text,(30,240),cv2.FONT_HERSHEY_TRIPLEX,2,(255,255,255))# calculated text format
        cv2.putText(blackboard,info,(30,440),cv2.FONT_HERSHEY_TRIPLEX,1,(0,255,255)) # info format
        if is_voice_on: #voice text format if on/off
            cv2.putText(blackboard,"Voice on",(450,440),cv2.FONT_HERSHEY_TRIPLEX,1,(255,127,0))
        else:
            cv2.putText(blackboard,"Voice off",(450,440),cv2.FONT_HERSHEY_TRIPLEX,1,(255,127,0))
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2) #image into rectangle
        res = np.hstack((img,blackboard))#stack horizontally
        cv2.imshow("Recognizing Gesture",res) #show the gesture
        cv2.imshow("thresh",thresh) #shoe the threshold images
        keypress = cv2.waitkey(1) #wait for the  sec to next image t show up
        if keypress == ord('q') or keypress == ord('t'): # if the key press q or t then come out of loop
            break 
        if keypress == ord('v') and is_voice_on: #if the key press v then voice on/off
            is_voice_on = False
        elif keypress = ord('v') and not is_voice_on:
            is_voice_on = True
    if keypress == ord('c'):  #if the key pressed c then it give the value 2
        return 2
    else:
        return 0                            

In [ ]:
def text_mode(cam):
    global is_voice_on
    text = "" #initialize the text
    word = "" #Initialize the word
    count_same_frame = 0 #intialize the number of same frame
    while True:
        img = cam.read()[1] #read the images
        img,contours,thresh = get_img_contour_thresh(img) #get the contours and thresh image 
        old_text = text #store old text into text
        if len(contours) > 0: #if length of contour is greater than 0
            contour = max(contours,key = cv2.contourArea) #then consider the contour with max area
            if cv2.contourArea > 10000: #if contour area greater than 10000
                text = get_pred_from_contour(contourArea) #text come after the max contour area is stored into text
                if old_text == text: #if old text is equal to text
                    count_same_frame += 1 #Increase the count of same frame
                else:
                    count_same_frame = 0 #else number of same frame is zero
                if count_same_frame > 20: #if number of same frame greater than 20
                    if len(text) == 1: #if length of text greater than 1
                        Thread(target = say_text,args=(text,)).start() #multiprocessing of text input 
                    word = word +text #then word is word + text
                    if word.startswith('I/Me '): #if  word start with given word
                        word = word.replace('I/Me ','I ') #replace it
                    elif word.endswith('I/Me '): #else ends with specified word
                        word = word.replace('I/Me ','me ')  #replace it accordingly
                    count_same_frame = 0
            elif cv2.contourArea < 10000: # if conour area less than 10000 then only say the text
                if word != '':
                    print = 'yolo'
                    say_text(text)
                    Thread(target=say_text,args=(word,)).start()
                text = ""
                word = ""
        else:
             if word != '':
                    print('yolo1')
                    say_text(text)
                    Thread(target = say_text,args=(word,)).start()
            text = ""
            word = ""
        blackboard = np.zeros((480,640,3),dtype = uint8)
        cv2.putText(blackboard,"Calculator Mode",(100,50),cv2.FONT_HERSHEY_TRIPLEX,1.5,(255,0,0))
        cv2.putText(blackboard,"Predicted Text-"+pred_text,(30,100),cv2.FONT_HERSHEY_TRIPLEX,1,(255,255,0))
        cv2.putText(blackboard,"operator ",+ operator,(30,140),cv2.FONT_HERSHEY_TRIPLEX,1,(255,255,127))
        cv2.putText(blackboard,word,(30,240),cv2.FONT_HERSHEY_TRIPLEX,2,(255,255,255))
        if is_voice_on:
            cv2.putText(blackboard,"Voice_on",(450,440),cv2.FONT_HERSHEY_tRIPLEX,1,(255,127,0))
        else:
            cv2.putText(blackboard,"Voice off",(450,440),cv2.FONT_HERSHEY_TRIPLEX,1,(255,127,0))
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        res = np.hstack("thresh",thresh)
        cv2.imshow("Recognizing gestures",res)
        cv2.imshow("thresh",thresh)
        keypress = cv2.waitkey(1)
        if keypress == ord('q') or keypress == ord('c'):
            break
        if keypress == ord('v') and is_voice_on:
            is_voice_on = False
        elif keypress == ord('v') and not is_voice_on:
            is_voice_on = True
    if keypress == ord('c'):
        return 2
    else:
        return 0

In [ ]:
def recognize():
    cam = cv2.VideoCapture(1) #capture image from index 1
    if cam.read()[0] == False: #if not able to capture then try index zero
        cam = cv2.VideoCapture(0)
    text = "" #initialize the text
    word = "" #Initialize the word
    count_same_frame = 0 #Initialize the number of same frame
    keypress = 1 #Initialise the keypress
    while True:
        if keypress == 1: #if keypress equal to 1 then text mode starts
            keypress = text_mode(cam)
        elif keypress == 2: #if key press equal to 2 then calculator mode starts
            keypress = calculator_mode(cam)
        else:
            break
keras_predict(model,np.zeros((50,50),dtype = np.uint8))
recognize()